# 아마존 세이지메이커에서 Llama 2 미세 조정하기

이번 예시에서는 아마존 세이지메이커를 사용해 [Llama 2](https://huggingface.co/meta-llama/Llama-2-70b-hf)를 미세 조정하는 방법을 배웁니다. [Llama 2](https://huggingface.co/meta-llama/Llama-2-70b-hf)는 [LLaMA](https://arxiv.org/abs/2302.13971)의 다음 버전입니다.

Llama 2는 LLaMA 모델과 비교해서 2T 토큰 이라는 더 많은 데이터를 학습했고 최대 4K 토큰의 콘텍스트 크기 윈도를 지원합니다. Llama 2에 대한 자세한 내용은 [블로그 게시물]에서 확인하세요.

이번 예시에서는 다음 내용을 배우게 됩니다.
1. 개발 환경 설정
2. 데이터 세트 적재 및 준비
3. 아마존 세이지메이커의 p4d.24xlarge 인스턴스를 사용해 Llama 7B 미세 조정하기
4. 아마존 세이지메이커에 미세 조정한 모델 배포

### Llama 2 접근하기

학습을 시작하기 전에 [Llama 2](https://huggingface.co/meta-llama/Llama-2-70b-hf)를 사용할 수 있도록 라이선스에 동의했는지 확인해야 합니다. 모델 페이지에서 `LLAMA 2 COMMUNITY LICENSE AGREEMENT` 항목에서 라이센스를 동의할 수 있습니다.

## 1. 개발 환경 설정

In [2]:
!pip install "transformers==4.44.0" "datasets[s3]==2.21.0" sagemaker --upgrade --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.4 which is incompatible.
spyder 5.3.3 requires ipython<8.0.0,>=7.31.1, but you have ipython 8.18.1 which is incompatible.
spyder 5.3.3 requires pylint<3.0,>=2.5.0, but you have pylint 3.0.2 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


Llama 2 자산에 접근하려면 허깅 페이스 계정에 로그인해야 합니다. 다음 명령을 실행해 로그인할 수 있습니다.

로컬 환경에서 세이지메이커를 사용하려면 세이지메이커에 필요한 권한이 있는 IAM 역할에 접근할 수 있어야 합니다. 이에 대한 자세한 내용은 [여기](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html)에서 확인할 수 있습니다.

In [3]:
import sagemaker
import boto3
sess = sagemaker.Session()

# 세이지메이커 세션 버킷 -> 데이터, 모델 및 로그 업로드에 사용
# 세션 버킷이 존재하지 않을 경우, 세이지메이커가 자동으로 버킷을 생성함
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # 버킷 이름이 주어지지 않은 경우 기본 버킷으로 설정함
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::079002598131:role/service-role/AmazonSageMaker-ExecutionRole-20220804T150518
sagemaker bucket: sagemaker-us-east-1-079002598131
sagemaker session region: us-east-1


## 데이터 세트 적재 및 준비

[InstructGPT 논문](https://arxiv.org/abs/2203.02155)에 설명된 여러 행동 범주(브레인스토밍, 분류, 폐쇄형 QA, 생성, 정보 추출, 개방형 QA, 요약)에 걸쳐 수천 명의 Databricks 직원들이 생성한 인스트럭션 수행 기록의 오픈 소스 데이터 세트인 [dolly](https://huggingface.co/datasets/databricks/databricks-dolly-15k)를 사용합니다.

```python
{
  "instruction": "World of Warcraft란 무엇인가?",
  "context": "",
  "response": "World of Warcraft는 대규모 온라인 멀티 플레이어 롤플레잉 게임입니다. 2004년에 Blizzard Entertainment에서 출시했습니다."
}
```

`samsum` 데이터 세트를 적재하기 위해 허깅 페이스 Datasets 라이브러리의 `load_dataset()` 메소드를 사용합니다.

In [4]:
from datasets import load_dataset
from random import randrange

# 허깅 페이스 허브에서 데이터 세트 적재
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")

print(f"dataset size: {len(dataset)}")
print(dataset[randrange(len(dataset))])

# dataset size: 15011

Found cached dataset json (/root/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-7427aa6e57c34282/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


dataset size: 15011
{'instruction': 'What is AWS EMR?', 'context': '', 'response': 'Amazon EMR (previously called Amazon Elastic MapReduce) is a managed cluster platform that simplifies running big data frameworks, such as Apache Hadoop and Apache Spark , on AWS to process and analyze vast amounts of data.', 'category': 'open_qa'}


모델을 인스트럭션 기반으로 미세 조정하려면 구조화된 예시를 인스트럭션으로 설명된 작업 모음으로 변환해야 합니다. 샘플을 받아 형식이 지정된 인스트럭션 문자열을 반환하는 `formatting_function`을 정의합니다.

In [5]:
def format_dolly(sample):
    instruction = f"### Instruction\n{sample['instruction']}"
    context = f"### Context\n{sample['context']}" if len(sample["context"]) > 0 else None
    response = f"### Answer\n{sample['response']}"
    # 모든 파트를 하나의 문자열로 결합합니다.
    prompt = "\n\n".join([i for i in [instruction, context, response] if i is not None])
    return prompt

formatting function을 임의의 예시로 테스트해 봅시다.

In [6]:
from random import randrange

print(format_dolly(dataset[randrange(len(dataset))]))

### Instruction
Do tomatoes belong in Louisiana-style gumbo?

### Answer
The answer is it depends.  In traditional Louisiana-style gumbo recipes from the area known for Cajun (Acadiana) cuisine, tomatoes are not commonly part of the recipe.  However, in the area known for Creole (New Orleans) cuisine tomatoes are commonly part of the recipe.


또한 샘플의 형식을 지정하는 것 외에도 더 효율적인 학습을 위해 여러 샘플을 하나의 시퀀스로 묶고자 합니다.

In [7]:
from transformers import AutoTokenizer

#model_id = "meta-llama/Llama-2-13b-hf" # 분할된 가중치, 게이트 처리
model_id = "NousResearch/Llama-2-7b-hf" # 게이트 처리 안함
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


주어진 길이의 시퀀스로 샘플을 묶은 다음 토큰화하는 몇 가지 보조 함수를 정의합니다.

In [8]:
from random import randint
from itertools import chain
from functools import partial

# 각 샘플에 프롬프트를 추가하기 위한 템플릿 데이터 세트
def template_dataset(sample):
    sample["text"] = f"{format_dolly(sample)}{tokenizer.eos_token}"
    return sample

# 각 샘플에 프롬프트 템플릿 적용
dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))
# 임의의 샘플 출력
print(dataset[randint(0, len(dataset))]["text"])

# 다음 배치에서 사용할 수 있도록 배치에서 남은 부분을 저장할 빈 리스트 생성
remainder = {"input_ids": [], "attention_mask": [], "token_type_ids": []}

def chunk(sample, chunk_length=2048):
    # 배치에서 남은 데이터를 다음 배치에서 재사용하기 위한 전역 변수 정의
    global remainder
    # 모든 텍스트를 연결하고 이전 배치에서 남은 부분 추가
    concatenated_examples = {k: list(chain(*sample[k])) for k in sample.keys()}
    concatenated_examples = {k: remainder[k] + concatenated_examples[k] for k in concatenated_examples.keys()}
    # 배치의 총 토큰 수 가져오기
    batch_total_length = len(concatenated_examples[list(sample.keys())[0]])

    # 배치에서 최대 청크 수 가져오기
    if batch_total_length >= chunk_length:
        batch_chunk_length = (batch_total_length // chunk_length) * chunk_length

    # 최대 길이의 청크로 분할
    result = {
        k: [t[i : i + chunk_length] for i in range(0, batch_chunk_length, chunk_length)]
        for k, t in concatenated_examples.items()
    }
    # 다음 배치를 위해 전역 변수에 남은 부분 추가
    remainder = {k: concatenated_examples[k][batch_chunk_length:] for k in concatenated_examples.keys()}
    # 레이블 준비
    result["labels"] = result["input_ids"].copy()
    return result


# 데이터 세트를 토큰화하고 청크로 나누기
lm_dataset = dataset.map(
    lambda sample: tokenizer(sample["text"]), batched=True, remove_columns=list(dataset.features)
).map(
    partial(chunk, chunk_length=2048),
    batched=True,
)

# 전체 샘플 수 출력
print(f"Total number of samples: {len(lm_dataset)}")

Loading cached processed dataset at /root/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-7427aa6e57c34282/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-a9e8aa13e41a7fc8.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-7427aa6e57c34282/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-9286c08bce910be1.arrow


### Instruction
Identify the bird from the list: Mouse, Balearic shearwater, Giraffe

### Answer
Balearic shearwater</s>


Loading cached processed dataset at /root/.cache/huggingface/datasets/databricks___json/databricks--databricks-dolly-15k-7427aa6e57c34282/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-75f16e9e4c95140b.arrow


Total number of samples: 1591


데이터 세트를 처리한 후 새로운 [파일 시스템 통합 기능](https://huggingface.co/docs/datasets/filesystems)을 사용해 데이터 세트를 S3에 업로드 할 예정입니다. `sess.default_bucket()`을 사용하고 있는데, 다른 S3 버킷에 데이터를 저장하고 싶다면 이 부분을 수정하세요. 이후의 학습 스크립트에서도 해당 S3 경로를 사용합니다.

In [9]:
# S3에 학습한 데이터 세트 저장
training_input_path = f's3://{sess.default_bucket()}/processed/llama/dolly/train'
lm_dataset.save_to_disk(training_input_path)

print("uploaded data to:")
print(f"training dataset to: {training_input_path}")

Saving the dataset (0/1 shards):   0%|          | 0/1591 [00:00<?, ? examples/s]

uploaded data to:
training dataset to: s3://sagemaker-us-east-1-079002598131/processed/llama/dolly/train


## 3. 아마존 세이지메이커에서 p4d.24xlarge 인스턴스를 사용해 Llama 7B 미세 조정하기

모델을 학습시키기 위한 스크립트([train.py](./scripts/train.py))를 준비했습니다.

아마존 세이지메이커 학습 작업을 생성하려면 허깅 페이스 Estimator가 필요합니다. Estimator는 아마존 세이지메이커의 학습 및 배포 작업을 처음부터 끝까지 처리합니다. Estimator는 일련의 과정에서 사용되는 인프라를 관리합니다.

세이지메이커는 필요한 모든 EC2 인스턴스를 자동으로 시작하고 관리하며, 적절한 허깅 페이스 컨테이너를 제공하고, 제공된 스크립트를 업로드하며, S3 버킷에서 데이터를 컨테이너의 `/opt/ml/input/data` 경로로 다운로드합니다. 그런 다음 학습 작업을 실행합니다.

In [10]:
import time
from sagemaker.huggingface import HuggingFace
from huggingface_hub import HfFolder

# 학습 작업 이름 정의
job_name = f'huggingface-lora-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'

# 학습 작업에 전달되는 하이퍼파라미터
hyperparameters ={
  'model_id': model_id,                             # 사전 학습된 모델
  'dataset_path': '/opt/ml/input/data/training',    # 세이지메이커가 학습 데이터 세트를 저장할 경로
  'epochs': 3,                                      # 학습 에포크 수
  'per_device_train_batch_size': 2,                 # 학습 시 사용되는 배치 크기
  'lr': 2e-4,                                       # 학습 중 사용되는 학습률
}

# Estimator 생성
huggingface_estimator = HuggingFace(
    entry_point          = 'train.py',        # 학습 스크립트
    source_dir           = 'scripts',         # 학습에 필요한 모든 파일이 포함된 디렉터리
    instance_type        = 'ml.p4d.24xlarge', # 학습 작업에 사용되는 인스턴스 유형
    instance_count       = 1,                 # 학습에 사용되는 인스턴스 수
    base_job_name        = job_name,          # 학습 작업 이름
    role                 = role,              # AWS 리소스에 접근하기 위해 학습 작업에서 사용되는 IAM 역할 (예: S3)
    volume_size          = 300,               # EBS 볼륨 크기 (GB 단위)
    transformers_version = '4.28',            # 학습 작업에 사용되는 transformers 버전
    pytorch_version      = '2.0',             # 학습 작업에 사용되는 PyTorch 버전
    py_version           = 'py310',           # 학습 작업에 사용되는 Python 버전
    hyperparameters      =  hyperparameters,  # 학습 작업에 전달되는 하이퍼파라미터
    environment          = { "HUGGINGFACE_HUB_CACHE": "/tmp/.cache" }, # 모델을 /tmp에 캐시하기 위한 환경 변수 설정
)

In [10]:
import time
from sagemaker.huggingface import HuggingFace
from huggingface_hub import HfFolder

# 학습 작업 이름 정의
job_name = f'huggingface-lora-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'

# 학습 작업에 전달되는 하이퍼파라미터
hyperparameters ={
  'model_id': model_id,                             # 사전 학습된 모델
  'dataset_path': '/opt/ml/input/data/training',    # 세이지메이커가 학습 데이터 세트를 저장할 경로
  'epochs': 3,                                      # 학습 에포크 수
  'per_device_train_batch_size': 2,                 # 학습 시 사용되는 배치 크기
  'lr': 2e-4,                                       # 학습 중 사용되는 학습률
}

# Estimator 생성
huggingface_estimator = HuggingFace(
    entry_point          = 'train.py',        # 학습 스크립트
    source_dir           = 'scripts',         # 학습에 필요한 모든 파일이 포함된 디렉터리
    instance_type        = 'ml.p4d.24xlarge', # 학습 작업에 사용되는 인스턴스 유형
    instance_count       = 1,                 # 학습에 사용되는 인스턴스 수
    base_job_name        = job_name,          # 학습 작업 이름
    role                 = role,              # AWS 리소스에 접근하기 위해 학습 작업에서 사용되는 IAM 역할 (예: S3)
    volume_size          = 300,               # EBS 볼륨 크기 (GB 단위)
    transformers_version = '4.28',            # 학습 작업에 사용되는 transformers 버전
    pytorch_version      = '2.0',             # 학습 작업에 사용되는 PyTorch 버전
    py_version           = 'py310',           # 학습 작업에 사용되는 Python 버전
    hyperparameters      =  hyperparameters,  # 학습 작업에 전달되는 하이퍼파라미터
    environment          = { "HUGGINGFACE_HUB_CACHE": "/tmp/.cache" }, # 모델을 /tmp에 캐시하기 위한 환경 변수 설정
)

In [10]:
import time
from sagemaker.huggingface import HuggingFace
from huggingface_hub import HfFolder

# 학습 작업 이름 정의
job_name = f'huggingface-lora-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'

# 학습 작업에 전달되는 하이퍼파라미터
hyperparameters ={
  'model_id': model_id,                             # 사전 학습된 모델
  'dataset_path': '/opt/ml/input/data/training',    # 세이지메이커가 학습 데이터 세트를 저장할 경로
  'epochs': 3,                                      # 학습 에포크 수
  'per_device_train_batch_size': 2,                 # 학습 시 사용되는 배치 크기
  'lr': 2e-4,                                       # 학습 중 사용되는 학습률
}

# Estimator 생성
huggingface_estimator = HuggingFace(
    entry_point          = 'train.py',        # 학습 스크립트
    source_dir           = 'scripts',         # 학습에 필요한 모든 파일이 포함된 디렉터리
    instance_type        = 'ml.p4d.24xlarge', # 학습 작업에 사용되는 인스턴스 유형
    instance_count       = 1,                 # 학습에 사용되는 인스턴스 수
    base_job_name        = job_name,          # 학습 작업 이름
    role                 = role,              # AWS 리소스에 접근하기 위해 학습 작업에서 사용되는 IAM 역할 (예: S3)
    volume_size          = 300,               # EBS 볼륨 크기 (GB 단위)
    transformers_version = '4.28',            # 학습 작업에 사용되는 transformers 버전
    pytorch_version      = '2.0',             # 학습 작업에 사용되는 PyTorch 버전
    py_version           = 'py310',           # 학습 작업에 사용되는 Python 버전
    hyperparameters      =  hyperparameters,  # 학습 작업에 전달되는 하이퍼파라미터
    environment          = { "HUGGINGFACE_HUB_CACHE": "/tmp/.cache" }, # 모델을 /tmp에 캐시하기 위한 환경 변수 설정
)

이제 `.fit()` 메서드를 사용해 S3 경로를 학습 스크립트에 전달함으로써 학습 작업을 시작할 수 있습니다.

In [11]:
# 업로드한 S3 URI를 사용해 데이터 입력 사전 정의
data = {'training': training_input_path}

# 업로드한 데이터 세트를 입력으로 사용해 학습 작업 시작
huggingface_estimator.fit(data, wait=True)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-lora-2024-01-01-19-55-37-2024-01-01-19-55-38-266


2024-01-01 19:55:38 Starting - Starting the training job
2024-01-01 19:55:38 Pending - Training job waiting for capacity......
2024-01-01 19:56:23 Pending - Preparing the instances for training.................................
2024-01-01 20:01:54 Downloading - Downloading input data...
2024-01-01 20:02:24 Downloading - Downloading the training image.....................
2024-01-01 20:06:05 Training - Training image download completed. Training in progress........bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-01-01 20:07:13,031 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-01-01 20:07:13,125 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-01-01 20:07:13,133 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-01-01 20:07:13,134 sagemaker_pytorch_container.training INFO     Invoking u